# Import des librairies

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

! Peut être nécessaire sur Windows. Il faut redémarrer le kernel Jupyter après installation !

In [ ]:
# !pip install --upgrade nbformat

# Extraction du fichier CSV à partir du fichier .zip brut
Le fichier CSV ayant une taille de 383,7 Mo, il est impossible de le stocker sur Github. En effet, Github bloque l'upload des fichiers de plus de 100 Mo. Afin de pouvoir reproduire l'analyse des données et le traitement effectué dans ce Notebook, nous allons donc utiliser le fichier au format .zip tel que récupéré sur Kaggle.
Si une copie locale des données au format CSV existe, alors cette copie est utilisée. Sinon, le fichier CSV est extrait lors de la première exécution du Notebook.

In [ ]:
if not os.path.exists("data/raw/btcusd_1-min_data.csv"):
    print("Fichier CSV inexistant, extraction à partir du fichier .zip ...")
    with zipfile.ZipFile("data/raw/btcusd_1-min_data_11_30_2025.zip","r") as zip_ref:
        zip_ref.extractall("data/raw/")
        print("Fichier CSV créé !")
else:
    print("Fichier CSV déjà existant, poursuite de l'exécution ...")


# Analyse préliminaire

Chargement des données CSV dans un DataFrame:

In [ ]:
df_bitcoin_raw = pd.read_csv("data/raw/btcusd_1-min_data.csv")

In [ ]:
df_bitcoin_raw.info()

In [ ]:
length_df_raw = len(df_bitcoin_raw)
print(length_df_raw)

Nous obtenons donc un DataFrame comportant 7 317 759 lignes, avec 6 colonnes qui sont: `Timestamp`, `Open`, `High`, `Low`, `Close` et `Volume`
Toutes ces colonnes sont pour l'instant de `dtypes: float(64)`.

## Statistiques descriptives du DataFrame

In [ ]:
df_bitcoin_raw.describe()

In [ ]:
df_bitcoin_raw.dtypes

## Identification des valeurs manquantes

In [ ]:
print(df_bitcoin_raw.isna().sum())

On peut observer qu'il n'y a aucune valeurs manquantes.

## Identifications des doublons
Il n'est pas pertinent d'étudier les doublons des colonnes numériques `Open`, `High`, `Low`, `Close` et `Volume`.
En effet, le Bitcoin peut très bien avoir eu le même prix ou volume plusieurs fois depuis que les données ont été collectées.
Nous allons cependant étudier la colonne `Timestamp`, représentant un instant T où les données ont été relevées.

In [ ]:
print(df_bitcoin_raw["Timestamp"].iloc[0])


On remarque que les valeurs contenues dans la colonne `Timestamp` sont au format Timestamp Unix, qui associe un nombre réel au temps mesuré depuis le 1er janvier 1970 à 00:00:00.
Ces valeurs sont donc supposées uniques (un instant T correspondant à un nombre réel unique), nous pouvons donc facilement vérifier l'existance de doublons.

In [ ]:
unique_timestamps = df_bitcoin_raw["Timestamp"].nunique()
print(f"Longueur du DataFame et nombre de Timestamp différent équivalent ? {unique_timestamps == length_df_raw}")
print(f"Nombre de lignes doublons: {df_bitcoin_raw.duplicated().sum()}")

Le nombre de valeurs uniques de Timestamp étant égal au nombre de lignes dans le DataFrame et n'ayant pas de lignes en double, nous pouvons estimer qu'il n'existe pas de lignes dupliquées dans les données utilisées.

## Conversion de "Timestamp" (float64, unix epoch time) en DateTime
Afin de poursuivre l'analyse sans modifier le DataFrame de base, nous utiliserons désormais une copie.

In [ ]:
df_bitcoin_modified = df_bitcoin_raw.copy()

La conversion du Timestamp en format lisible nous permet une meilleure analyse des données. Il est nécessaire de convertire ce Timestamp de `float` à `int`. Ce timestamp correspond au format Unix équivalent aux secondes, ce qui doit être passé en argument.

In [ ]:
df_bitcoin_modified["Timestamp"] = pd.to_datetime(df_bitcoin_modified["Timestamp"].astype(int), unit="s")

In [ ]:
df_bitcoin_modified

In [ ]:
df_bitcoin_modified.dtypes

## Analyse de l'existence de périodes manquantes
Les données issues de Timestamp étant désormais dans un format lisible et reconnaissable par Pandas, nous pouvons donc vérifier si des périodes de temps sont manquantes.

Pour cela, nous allons trier les dates dans un ordre ascendant (dans les cas où les lignes ne sont pas correctement "rangées"), puis nous pourrons récupérer les valeurs minimum et maximum afin de créer une `Serie` temporelle équivalente.

En comparant notre colonne `Timestamp` à cette série, nous pourrons identifier les périodes temporelles manquantes si elles existent.

In [ ]:
serie_timestamp = df_bitcoin_modified['Timestamp'].sort_values(ascending=True)

In [ ]:
first_timestamp = serie_timestamp.iloc[0]
last_timestamp = serie_timestamp.iloc[-1]
print(f"Première valeur Timestamp : {first_timestamp}")
print(f"Dernière valeur Timestamp : {last_timestamp}")

Créons maintenant la série temporelle permettant de vérifier les différences, en utilisant les valeurs trouvées.

In [ ]:
serie_timestamp_difference = pd.Series(pd.date_range(start=first_timestamp, end=last_timestamp, freq='min'))
print(serie_timestamp_difference.head())

Vérifions désormais la différence.

In [ ]:
missing_periods = serie_timestamp_difference[~serie_timestamp_difference.isin(serie_timestamp)]
if len(missing_periods) > 0:
    print(f"Il y a {len(missing_periods)} périodes manquantes au total")

    # Vérifions si ces périodes manquantes sont réunies en un seul 'bloc' ou si elles sont dispersées
    missing_sorted = missing_periods.sort_values(ascending=True) # Tri ascendant pour s'assurer de la cohérence

    # On sait que l'intervalle attendu correspond à 1 minute, puisque notre serie_timestamp_difference a été créé avec une
    # fréquence d'1 minute
    expected_interval = pd.Timedelta(minutes=1)

    # Si la différence est égale à 1 minute, les périodes manquantes sont dans le même 'bloc'
    differences = missing_sorted.diff()

    # On calcule le nombre de 'blocs' où l'intervalle ne correspond pas à la valeur attendue
    num_blocks = (differences != expected_interval).sum()
    print(f"Il y a {num_blocks} blocs de temps manquants")

    # Nous cherchons maintenant à identifier les débuts et fins des blocs de temps manquants
    block_starts = missing_sorted.loc[differences != expected_interval].index
    for i, start_index in enumerate(block_starts):
        # On récupère le timestamp de début du bloc actuel
        start_timestamp = missing_sorted.loc[start_index]

        # On vérifie s'il y a un bloc suivant, `-1` car l'index débute à `0`
        if i < len(block_starts) - 1:
            next_start = block_starts[i + 1]
            # Le timestamp de fin est celui juste avant le début du prochain bloc
            end_index = missing_sorted.index[missing_sorted.index.get_loc(next_start) - 1]
            end_timestamp = missing_sorted.loc[end_index]
        else:
            # Pour le dernier bloc, la fin est la dernière période manquante
            end_timestamp = missing_sorted.iloc[-1]
            print(f"Bloc {i+1}: de {start_timestamp} à {end_timestamp}")
else:
    print("Aucune période manquante détectée")


Nous observons donc un bloc de 1160 minutes consécutives pendant lesquelles les données n'ont pas été relevées.

Cela peut s'expliquer par une panne de la plateforme d'échange de cryptomonnaies utilisée pour collecter les données ou un problème technique rencontré par l'outil utilisé pour collecter les données.

In [ ]:
missing_pct = len(missing_periods) / unique_timestamps * 100
print(f"Ces périodes manquantes correspondent à {round(missing_pct, 5)}% des données totales")

L'existence de ces périodes manquantes ne posera plus de problèmes plus tard, lorsque nous réaliserons des agrégations temporelles.

## Utilisation de Timestamp comme index
Afin de pouvoir analyser plus finement les données, nous utiliserons les valeurs de la colonne `Timestamp` comme index de notre DataFrame.

In [ ]:
df_bitcoin_modified.set_index("Timestamp", inplace=True)

## Interpolation linéaire afin de combler les 1160 minutes manquantes
L'interpolation linéaire permet de remplir les valeurs manquantes à l'aide d'une fonction affine estimé à partir des données connues du DataFrame.

In [ ]:
df_bitcoin_modified= df_bitcoin_modified.resample('1min').interpolate(method='linear')

Nous pouvons vérifier si les valeurs manquantes ont bien été comblées.

In [ ]:
print(f"Nombre de lignes du DataFrame brut: {length_df_raw}")
print(f"Nombre de lignes du DataFrame après interpolation linéaire: {len(df_bitcoin_modified)}")
print(f"Différence entre les deux valeurs: {len(df_bitcoin_modified) - length_df_raw}")

Nous obtenons bien 1160 nouvelles lignes.

# Analyse univariée
## Analyse de la distribution
Nous allons dans un premier temps analyser la forme de la distribution des données.

Pour cela nous utiliserons une fonction retournant un DataFrame identifiant les valeurs `skew()` et `kurt()` pour chaque colonne de notre DataFrame.

In [ ]:
def identify_distribution_to_df(df: DataFrame) -> DataFrame:
    """
    Analyse les colonnes numériques d'un DataFrame et identifie leurs valeurs skew
    et kurt. Retourne les valeurs dans un nouveau DataFrame.\n
    skew: Asymétrie (0 = symétrique)\n
    kurt: Aplatissement (>3 = queues épaisses)
    :param df: DataFrame
    :type df: pandas.DataFrame
    :return: DataFrame contenant les noms des colonnes d'entrée, leurs valeurs
     skew et kurt.
    :rtype: tuple
    """
    distribution_list = []
    for series_name, series in df.select_dtypes(include=np.number).items():
        skew = series.skew()
        kurt = series.kurt()

        column_dict = {"Column": series_name, "skew": skew, "kurt": kurt}
        distribution_list.append(column_dict)

    df_stats = pd.DataFrame(distribution_list)

    return df_stats


In [ ]:
df_bitcoin_stats = identify_distribution_to_df(df_bitcoin_modified)
df_bitcoin_stats

Nous pouvons observer une forte asymétrie (valeur `skew`) pour chacune de nos colonnes, ainsi que la présence d'une queue à droite (valeur `skew` > 0) pour chaque colonne étudiée.

Nous pouvons observer la forme de la distribution à l'aide de Violin plots. Un échantillonnage aléatoire est utilisé afin de n'étudier que 100000 (valeur `sample_size`) points au lieu de plus de 7.3 millions, accélerant grandement l'affichage des visualisations.

Un `random_state` est défini à `42` pour la reproductibilité de l'échantillonnage.

In [ ]:
sample_size = 100_000
df_sample_violins = df_bitcoin_modified.sample(n=sample_size, random_state=42)

fig, axes = plt.subplots(1, 5, figsize=(18, 6))

for i, col in enumerate(['Open', 'High', 'Low', 'Close', 'Volume']):
    sns.violinplot(y=df_sample_violins[col], ax=axes[i])
    axes[i].set_title(f'Violin plot de {col}\n(échantillon: {sample_size:,} points)')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.show()


## Calcul des Z-Scores
Afin d'identifier les outliers (valeurs aberrantes), nous allons calculer le Z-score correspondant à chaque valeur de notre dataset.

In [ ]:
def calculate_z_scores(df: DataFrame) -> DataFrame:
    """
    Calcule les z-scores pour chaque colonne d'un DataFrame.
    Le z-score indique à combien d'écarts-types une valeur se situe de la moyenne.\n
    Formule: z = (x - moyenne) / écart-type\n
    Un z-score de 0 = valeur égale à la moyenne\n
    Un z-score de 2 = valeur à 2 écarts-types au-dessus de la moyenne
    Un z-score > 3 = outlier potentiel
    :param df: DataFrame contenant les données à utiliser
    :type df: pandas.DataFrame
    :return: DataFrame contenant les z-scores pour chaque colonne
    :rtype: pandas.DataFrame
    """
    return (df - df.mean()) / df.std()

In [ ]:
df_z_score = calculate_z_scores(df_bitcoin_modified)
print(df_z_score.shape)

Nous pouvons donc maintenant identifier les valeurs aberrantes, quand la valeur aboslue d'un z-score est supeérieur à 3.

In [ ]:
outliers_mask = (abs(df_z_score) > 3).any(axis=1)
outliers_z_score = df_bitcoin_modified.loc[outliers_mask]

print(f"Nombre de outliers détectés: {len(outliers_z_score)}")

Etudions et visualisons la distribution de ces outliers.

In [ ]:
outliers_z_score_distribution = identify_distribution_to_df(outliers_z_score)
print(outliers_z_score_distribution)

Les colonnes Open, High, Low et Close présentent de légères asymètries à gauche (`skew` négatif) et des queues plus fines que la normale (`kurt` négatif).

Mais la colonne Volume présente une très forte asymétrie à droite (`skew` d'environ 6.64) ainsi qu'une queue épaisse, avec beaucoup d'extrêmes (`kurt` d'environ 134.84)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, col in enumerate(outliers_z_score.columns):
    skew_val = outliers_z_score_distribution.loc[outliers_z_score_distribution['Column'] == col, 'skew'].values[0]
    kurt_val = outliers_z_score_distribution.loc[outliers_z_score_distribution['Column'] == col, 'kurt'].values[0]

    sns.violinplot(y=outliers_z_score[col], ax=axes[i])
    axes[i].set_title(f'{col}\nskew={skew_val:.2f}, kurt={kurt_val:.2f}')
plt.tight_layout()
plt.show()

Ces outliers sont significatifs de période de comportement extrême du prix du Bitcoin (période de crash, bullrun, etc...).

Comme ces valeurs correspondent à des évènements bien réels, la meilleure stratégie à adopter est de les conserver.

## Calcul des quartiles et interprétation
Calculons désormais les quartiles, les écarts interquartiles et les bornes inférieurs et supérieurs de nos valeurs.

In [ ]:
def identify_quartiles(df: DataFrame) -> DataFrame:
    """
    Calcule les quartiles et les bornes inférieures et supérieures pour chaque colonne d'un DataFrame.\n
    Q1: Premier quartile (25e percentile)\n
    Q3: Troisième quartile (75e percentile)\n
    IQR: Ecart interquartile (Q3 - Q1)\n
    Borne inférieure: Q1 - 1.5 × IQR (valeurs en dessous = outliers)\n
    Borne supérieure: Q3 + 1.5 × IQR (valeurs au-dessus = outliers)
    :param df: DataFrame contenant les données à analyser
    :type df: pandas.DataFrame
    :return: DataFrame contenant les quartiles et bornes pour chaque colonne
    :rtype: pandas.DataFrame
    """
    quartiles_list = []

    for series_name, series in df.items():
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower_fence = q1 - 1.5 * iqr
        upper_fence = q3 + 1.5 * iqr

        column_dict = {
            "Colonne": series_name,
            "Q1": q1,
            "Q3": q3,
            "IQR": iqr,
            "Borne inférieure": lower_fence,
            "Borne supérieure": upper_fence
        }
        quartiles_list.append(column_dict)

    df_quartiles = pd.DataFrame(quartiles_list)

    return df_quartiles

In [ ]:
df_quartiles = identify_quartiles(df_bitcoin_modified)
df_quartiles

Les colonnes Open, High, Low et Close présentent de très larges écarts interquartiles (`IQR`) d'environ 30230, ce qui signifie qu'environ 50% de nos données de prix se situent entre 443 et 30,680$. Cela montre une très forte volatilité du prix du Bitcoin sur la période mesurée.

Si on souhaitait utiliser l'IQR afin d'identifier les outliers, alors tout prix supérieur à environ 76,000$ serait considéré comme anormal, ce qui ne respecte pas la tendance globale du prix du Bitcoin:

In [ ]:
mean_upper_fence = df_quartiles.loc[df_quartiles['Colonne'].isin(['Open', 'High', 'Low', 'Close']), 'Borne supérieure'].mean()

plt.figure(figsize=(14, 6))
plt.plot(df_bitcoin_modified.index, df_bitcoin_modified['Close'], label='Prix Close', linewidth=0.5)
plt.axhline(y=mean_upper_fence, color='red', linestyle='--', linewidth=1, label=f'Borne sup. moyenne: {mean_upper_fence:.2f}$')
plt.xlabel('Date')
plt.ylabel('Prix (USD)')
plt.title('Prix du Bitcoin avec borne supérieure moyenne des quartiles')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Nous pouvons donc en déduire que l'utilisation de l'IQR n'est pas adaptée pour identifier les outliers / anomalies de prix du Bitcoin.

Quant au volume, une borne supérieure d'environ 7.21 et un IQR d'environ 2.88 semblent être pertinent pour identifier les anomalies de volume. Nous pouvons le vérifier à l'aide d'une visualisation:

In [ ]:
volume_upper_fence = df_quartiles.loc[df_quartiles['Colonne'] == 'Volume', 'Borne supérieure'].values[0]

# Nécessaire pour éviter de devoir tracer 7.3 millions de points
df_hourly_volume = df_bitcoin_modified['Volume'].resample('h').mean()

plt.figure(figsize=(14, 6))
plt.plot(df_hourly_volume.index, df_hourly_volume, label='Volume (horaire)', linewidth=0.5, alpha=0.7)
plt.axhline(y=volume_upper_fence, color='red', linestyle='--', linewidth=2, label=f'Borne sup.: {volume_upper_fence:.2f}')
plt.xlabel('Date')
plt.ylabel('Volume (échelle log)')
plt.yscale('log')
plt.title('Volume du Bitcoin, échelle logarithmique')
plt.xlim(df_hourly_volume.index.min(), df_hourly_volume.index.max())
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Sur le graphique ci-dessus, les valeurs au-dessus de la ligne rouge (représentant la borne supérieure du volume) peuvent être considérée comme des anomalies de volume.

# Feature Engineering
Maintenant que nous avons analysé en détail les distributions et statistiques de nos données, nous pouvons calculer de nouvelles variables intéressantes.

Commençons d'abord par standardiser nos colonnes initiales, en les convertissant en minuscules.

In [ ]:
df_bitcoin_modified.columns = df_bitcoin_modified.columns.str.lower()
print(df_bitcoin_modified.columns)

## `returns`, taux d'évolution du prix par rapport au prix précédent

In [ ]:
df_bitcoin_modified['returns'] = df_bitcoin_modified['close'].pct_change()
df_bitcoin_modified['returns'].head()

La première valeur `returns` est un `NaN`, car il n'y a pas de valeur précédente. Nous pouvons transformer cette valeur en `0.0`.

In [ ]:
df_bitcoin_modified['returns'] = df_bitcoin_modified['returns'].fillna(0.0)
df_bitcoin_modified['returns'].head()

## `volatility`, volatilité sur la dernière heure
La volatilité est calculée en fonction de l'écart-type des valeurs `returns` sur une fênetre roulante de 60 périodes, soit 60 minutes.

In [ ]:
df_bitcoin_modified['volatility'] = df_bitcoin_modified['returns'].rolling(60).std()
df_bitcoin_modified['volatility'].iloc[54:65]

Les 60 premières valeurs sont `NaN`, car le nombre de période temporelle nécessaire au calcul n'est pas encore atteint. Les données seront agrégées plus tard, ce qui résolvera tous problèmes éventuels liés à ces valeurs.

## `price_range`, intervalle entre le prix `high` et le prix `low`

In [ ]:
df_bitcoin_modified['price_range'] = df_bitcoin_modified['high'] - df_bitcoin_modified['low']

# Echantillon aléatoire, avec seed
df_bitcoin_modified['price_range'].sample(n=5, random_state=42)

## `close_ma_60`, moyenne mobile sur 60 période du prix `close`

In [ ]:
df_bitcoin_modified['close_ma_60'] = df_bitcoin_modified['close'].rolling(60).mean()
df_bitcoin_modified['close_ma_60'].iloc[54:65]

Ici encore, les 60 premières valeurs sont `NaN`, car il n'y a pas assez de période pour les calculs. Une fois agrégées, ces valeurs disparaitront.

## `volume_ma_60`, moyenne mobile sur 60 période du volume

In [ ]:
df_bitcoin_modified['volume_ma_60'] = df_bitcoin_modified['volume'].rolling(60).mean()
df_bitcoin_modified['volume_ma_60'].iloc[54:65]

Idem pour cette nouvelle colonne.

In [ ]:
df_bitcoin_modified['volume_ma_60'].sample(n=5, random_state=42)


## `volume_log`, afin de réduire la valeur `skew()` de `volume`
Comme vu précédement, la valeur `skew()` de la colonne volume était très importante:

In [ ]:
df_bitcoin_stats.loc[df_bitcoin_stats['Column'] == 'Volume', 'skew']

Une échelle logarithmique permettera de grandement réduire cette valeur:

In [ ]:
df_bitcoin_modified['volume_log'] = np.log(df_bitcoin_modified['volume'] + 1)
print(f"Valeur skew() de volume_log: {df_bitcoin_modified['volume_log'].skew()}")

## Résumé des nouvelles colonnes

In [ ]:
df_bitcoin_modified.columns

# Export du DataFrame modifieé vers un nouveau fichier Parquet
Afin de gagner du temps pour de potentielles futures analyses ou pour une meilleure reproductibilité, nous pouvons maintenant exporter le DataFrame `df_bitcoin_modified` sous format Parquet, plus performant que CSV pour de grands jeux de données.

Il est cependant impossible de mettre en ligne ces données sur Github, le fichier étant trop volumineux.

In [ ]:
df_bitcoin_modified.to_parquet('data/clean/bitcoin_1min_clean.parquet', engine='fastparquet', compression='zstd')
print(f"DataFrame exporté en Parquet ...")

# Agrégations temporelles
Afin de procéder à des agrégations temporelles dans les meilleures conditions, nous pouvons créer une fonction dédiée nous permettant de recalculer les Features de la section précédente et de manipuler les variables existantes en un ensemble cohérent.

In [ ]:
def aggregate_ohlcv(df, freq, ma_windows):
    """
    Agrège correctement des données OHLCV.
    :param df: DataFrame avec colonnes open, high, low, close, volume
    :type df: pandas.DataFrame
    :param freq : fréquence ('h', 'D', 'ME', 'W', etc.) d'agrégation voulue
    :type freq: str
    :param ma_windows: fenêtres pour moyennes mobiles (ex: [7, 30, 90] pour 7j, 30j ou 90j)
    :type ma_windows: list
    :return: DataFrame agrégé avec features dérivées
    :rtype: pandas.DataFrame
    """

    df_agg = df.resample(freq).agg({
        # open doit correspondre à la première valeur open de la fréquence d'agrégation voulue
        'open': 'first',

        # high doit être la plus grande valeur high sur la même période
        'high': 'max',

        # low est la valeur low la plus petite sur la même période
        'low': 'min',

        # close correspond ici à la dernière valeur enregistrée pour close
        'close': 'last',

        # les volumes de la période d'agrégation doivent être additionnés
        'volume': 'sum'
    })

    # Features Engineering
    df_agg['returns'] = df_agg['close'].pct_change()
    df_agg['price_range'] = df_agg['high'] - df_agg['low']

    for window in ma_windows:
        df_agg[f'volatility_{window}'] = df_agg['returns'].rolling(window).std()
        df_agg[f'close_ma_{window}'] = df_agg['close'].rolling(window).mean()
        df_agg[f'volume_ma_{window}'] = df_agg['volume'].rolling(window).mean()


    df_agg['volume_log'] = np.log(df_agg['volume'] + 1)

    return df_agg

Cette fonction nous permet de définir une liste de périodes temporelles à utiliser en temps que fenêtre roulante, et nous pouvons calculer les mêmes valeurs dérivées que pour notre DataFrame `df_bitcoin_modified` mais sur des périodes agrégées.

In [ ]:
# Série agrégée 1 heure, avec fenêtre de 24h et d'une semaine
df_bitcoin_1_hour = aggregate_ohlcv(df_bitcoin_modified, 'h', ma_windows=[24, 168])
print(f"DataFrame horaire: {len(df_bitcoin_1_hour):,} lignes")

# Série agrégée 1 jour, avec fenêtre d'une semaine, un mois et trois mois
df_bitcoin_1_day = aggregate_ohlcv(df_bitcoin_modified, 'D', ma_windows=[7, 30, 90])
print(f"DataFrame journalier: {len(df_bitcoin_1_day):,} lignes")

# Série agrégée 1 mois, avec fenêtre de trois mois et d'un an
df_bitcoin_1_month = aggregate_ohlcv(df_bitcoin_modified, 'ME', ma_windows=[3, 12])
print(f"DataFrame mensuel: {len(df_bitcoin_1_month):,} lignes")

Statistiques descriptives pour l'agrégation 1 heure:

In [ ]:
df_bitcoin_1_hour.describe()

Statistiques descriptives pour l'agrégation 1 jour:

In [ ]:
df_bitcoin_1_day.describe()

Statistiques descriptives pour l'agrégation 1 mois:

In [ ]:
df_bitcoin_1_month.describe()

# Visualisation interactive de la courbe Close
En utilisant nos agrégations, nous pouvons désormais créer des visualisations plus claires qu'en utilisant les données collectées chaque minute. En effet, le grand nombre de point à placer dans les visualisations rend les graphiques difficiles à interpréter. Les agrégations permettent de résoudre ce problème.

## Visualisation interactive de l’évolution du prix dans le temps (courbe Close) avec Plotly Express

In [ ]:
fig = px.scatter(df_bitcoin_1_hour, x=df_bitcoin_1_hour.index, y="close", title="Prix de clôture du Bitcoin (horaire)")
fig.update_traces(marker=dict(size=2))
fig.update_xaxes(title="Date")
fig.update_yaxes(title="Prix (USD)",tickformat="$,.0f", separatethousands=True)
fig.show()

## Graphique en bougie du Bitcoin (courbe Close) avec Plotly Graph Objects

In [ ]:
fig = go.Figure(data=[
    go.Candlestick(
        x=df_bitcoin_1_day.index,
        open=df_bitcoin_1_day["open"],
        high=df_bitcoin_1_day["high"],
        low=df_bitcoin_1_day["low"],
        close=df_bitcoin_1_day["close"]
    )]
)

y_min = df_bitcoin_1_day["low"].min()
y_max = df_bitcoin_1_day["high"].max()
fig.update_layout(
    yaxis=dict(
        range=[y_min, y_max],
        tickformat="$,.0f",
        separatethousands=True,
        title="Prix (USD)"
    ),
    xaxis=dict(
        title="Date",
        rangeslider=dict(
            visible=True,
            thickness=0.05
        )
    ),
    height=800,
    title="Prix de clôture du Bitcoin (journalier)"
)

fig.show()


# Identification des périodes de Bull Run et de Crash
Afin d'identifier les différentes tendances de marché de manière systèmatique et reproductible, nous pouvons définir une classe et des méthodes utilisant des fenêtres mobiles, des maximums et minimums locaux ainsi que des seuils de gain et de perte sur une période définie.

Afin d'établir cette méthode, nous nous sommes inspirés des analyses suivantes :
- https://blockworks.co/news/bitcoin-bull-market-drawdowns
- https://www.kucoin.com/learn/crypto/the-history-of-bitcoin-bull-runs-and-crypto-market-cycles

In [ ]:
class BitcoinTrendDetector:
    """
    Analyse les données de prix du Bitcoin pour identifier les tendances du marché, telles que les crashs
    et les bull runs, en fonction de seuils et de durées prédéfinis.

    :ivar df: DataFrame contenant le jeu de données de prix du Bitcoin
    :type df: pandas.DataFrame
    :ivar price_col: Nom de la colonne de prix à analyser
    :type price_col: str
    :ivar trends: DataFrame résumant les périodes de tendance identifiées, incluant les dates de début et de fin, la durée et le rendement
    :type trends: pandas.DataFrame
    """

    def __init__(self, df, price_col="close"):
        """
        Initialise l'objet

        :param df: DataFrame contenant les données de prix du Bitcoin
        :type df: pandas.DataFrame
        :param price_col: Nom de la colonne de prix à analyser, par defaut "close"
        :type price_col: str
        """
        self.df = df.copy()
        self.price_col = price_col
        self.trends = None

    def identify_trends(
            self,
            min_crash_timeframe: str,
            min_bullrun_timeframe: str,
            crash_threshold: float,
            bullrun_threshold: float,
            window: str):
        """
        Identifie les tendances du marché (crash, bullrun, neutral) en fonction de seuils et durées spécifiés.

        :param min_crash_timeframe: Durée minimale pour qu'un crash soit considéré valide ('7D' pour 7 jours)
        :type min_crash_timeframe: str
        :param min_bullrun_timeframe: Durée minimale pour qu'un bull run soit considéré valide ('30D' pour 30 jours)
        :type min_bullrun_timeframe: str
        :param crash_threshold: Seuil de perte pour identifier un crash (-0.30 pour -30%)
        :type crash_threshold: float
        :param bullrun_threshold: Seuil de gain depuis le plus bas pour identifier un bull run (1.0 pour +100%)
        :type bullrun_threshold: float
        :param window: Fenêtre pour calculer les maximums et minimums roulants ('180D' pour 180 jours)
        :type window: str
        :return: DataFrame contenant les périodes de tendance identifiées avec leurs caractéristiques
        :rtype: pandas.DataFrame
        """
        df = self.df.copy()
        price = df[self.price_col]

        # Trouve le maximum local sur la periode etudiee
        rolling_max = price.rolling(window=window, min_periods=1).max()
        # Calcule le drawdown (perte) depuis le maximum local
        drawdown = (price - rolling_max) / rolling_max

        # Trouve le minimum local sur la periode etudiee
        rolling_min = price.rolling(window=window, min_periods=1).min()
        # Calcule le gain depuis le minimum local
        gain_from_low = (price - rolling_min) / rolling_min

        # Calcule le changement de prix sur la periode etudiee
        price_change = price.pct_change(periods=pd.Timedelta(window).days)

        # Nouvelle serie initialisee en trend 'neutral'
        trend = pd.Series('neutral', index=df.index)

        # On identifie les periodes de perte superieures a notre seuil => crash + changement de prix negatif
        crash_mask = (drawdown <= crash_threshold) & (price_change < 0)
        trend[crash_mask] = 'crash'

        # On identifie les periodes de gain superieures a notre seuil => bull run + changement de prix positif
        bull_mask = (gain_from_low >= bullrun_threshold) & (price_change > 0)
        trend[bull_mask] = 'bullrun'

        # Applique le filtre de durée minimale pour les crashs et bull runs
        trend = self._apply_duration_filter(trend, 'crash', min_crash_timeframe)
        trend = self._apply_duration_filter(trend, 'bullrun', min_bullrun_timeframe)

        # Ajoute les colonnes de tendance et métriques au DataFrame
        df['trend'] = trend
        df['drawdown'] = drawdown
        df['gain_from_low'] = gain_from_low
        df['price_change'] = price_change

        self.df = df
        self.trends = self._extract_trend_periods()

        return self.trends

    @staticmethod
    def _apply_duration_filter(trend_series, trend_type, min_duration):
        """
        Filtre les périodes de tendance en fonction d'une durée minimale.

        :param trend_series: Série contenant les tendances identifiées
        :type trend_series: pandas.Series
        :param trend_type: Type de tendance à filtrer ('crash' ou 'bullrun')
        :type trend_type: str
        :param min_duration: Durée minimale requise ('7D')
        :type min_duration: str
        :return: Série de tendances filtrée
        :rtype: pandas.Series
        """
        result = trend_series.copy()

        # Permet la comparaison de temps
        min_delta = pd.Timedelta(min_duration)

        # Identifie les périodes correspondantes au type de tendance spécifié
        is_trend = trend_series == trend_type
        trend_changes = is_trend.astype(int).diff().fillna(0)

        # Trouve les points de début et de fin de chaque période
        starts = trend_series.index[trend_changes == 1]
        ends = trend_series.index[trend_changes == -1]

        # Vérifie la durée de chaque période et filtre celles trop courtes
        for start, end in zip(starts, ends):
            duration = end - start
            if duration < min_delta:
                result.loc[start:end] = 'neutral'

        return result

    def _extract_trend_periods(self):
        """
        Extrait les périodes de tendance du DataFrame.

        :return: DataFrame contenant les informations sur chaque période de tendance (type, dates, durée, prix de début/fin, rendement)
        :rtype: pandas.DataFrame
        """
        trend = self.df['trend']

        # On compare chaque valeur a la valeur precedente, ce qui retourne un bool, convertit en 0 si False et 1 si True
        trend_changes = (trend != trend.shift()).astype(int)
        price_col = self.df[self.price_col]

        periods = []
        current_trend = trend.iloc[0]
        start_date = trend.index[0]

        # Parcourt toutes les données pour identifier les changements de tendance
        for i in range(1, len(trend)):
            if trend_changes.iloc[i]:
                # Si la tendance précédente n'était pas neutre, on l'enregistre
                if current_trend != 'neutral':
                    periods.append({
                        'trend': current_trend,
                        'start': start_date,
                        'end': trend.index[i - 1],
                        'duration': trend.index[i - 1] - start_date,
                        'start_price': price_col.loc[start_date],
                        'end_price': price_col.loc[trend.index[i - 1]]
                    })
                current_trend = trend.iloc[i]
                start_date = trend.index[i]

        # Traite la dernière période si elle n'est pas neutre
        if current_trend != 'neutral':
            periods.append({
                'trend': current_trend,
                'start': start_date,
                'end': trend.index[-1],
                'duration': trend.index[-1] - start_date,
                'start_price': price_col.loc[start_date],
                'end_price': price_col.iloc[-1]
            })

        periods_df = pd.DataFrame(periods)
        # Calcule le rendement en pourcentage pour chaque période
        if len(periods_df) > 0:
            periods_df['return'] = (periods_df['end_price'] / periods_df['start_price'] - 1) * 100

        return periods_df


Les seuils `crash_threshold=-0.20` et `bullrun_threshold=0.20` sont définis de façon à respecter les standars de l'industrie :

- https://www.investopedia.com/terms/b/bullmarket.asp
- https://www.investopedia.com/terms/b/bearmarket.asp

In [ ]:
detector = BitcoinTrendDetector(df_bitcoin_1_day)
trends = detector.identify_trends(
    min_crash_timeframe='30D',
    min_bullrun_timeframe='30D',
    crash_threshold=-0.20,
    bullrun_threshold=0.20,
    window='90D'
)

In [ ]:
trends

Maintenant que nous avons identifieé nos périodes de Bull run et de Crash, nous pouvons les tracer (avec une échelle de prix logarithmique pour une meilleure clartée):

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(detector.df.index, detector.df[detector.price_col], color='black', linewidth=1.5, label='Prix du Bitcoin')

# Permet d'éviter d'avoir plusieurs fois le même label
bullrun_label_flag = False
crash_label_flag = False

for _, period in detector.trends.iterrows():
    if period['trend'] == 'bullrun':
        label = 'Bullrun' if not bullrun_label_flag else ''
        ax.axvspan(period['start'], period['end'], alpha=0.3, color='green', label=label)
        bullrun_label_flag = True
    elif period['trend'] == 'crash':
        label = 'Crash' if not crash_label_flag else ''
        ax.axvspan(period['start'], period['end'], alpha=0.3, color='red', label=label)
        crash_label_flag = True

ax.set_yscale('log')
ax.set_xlabel('Date', fontweight='bold')
ax.set_ylabel('Prix (USD)', fontweight='bold')
ax.set_title('Périodes de Bull Run et de Crash du Bitcoin', fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

Cette classe semble détecter avec précisions les Bull runs et Crashes passés. Il ne s'agit pas d'une méthode fiable pour déterminer les trends à venir, en raison de l'utilisation de périodes temporelles importantes.

On peut notamment remarquer des phases de Bull Run correspondant à la période 2017 - 2018, suivi d'une phase de plusieurs crashs. On observe un crash moyen début 2020, qui correspond à la crise COVID-19.

En somme, le cours du Bitcoin est généralement haussier sur le long terme.

# Volume d'échange au cours du temps

In [ ]:
fig = px.line(df_bitcoin_1_hour, x=df_bitcoin_1_hour.index, y="volume", title="Volume d'échange du Bitcoin (horaire)")
fig.update_traces(marker=dict(size=2))
fig.update_xaxes(title="Date")
fig.update_yaxes(title="Volume",separatethousands=True)
fig.show()

On remarque un pic de volume en février 2014. Cela correspond à un évènement marquant de l'histoire du Bitcoin, le piratage de la plateforme d'échange de cryptomonnaies Mt. Gox, qui causa le vol de plus de 744 000 Bitcoins.

# Analyse de la volatilité et variation relative
Nous allons analyser la volatilité pour les agrégations temporelles `df_bitcoin_1_day` et `df_bitcoin_1_hour`:

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

ax1.plot(df_bitcoin_1_hour.index, df_bitcoin_1_hour['volatility_24'],color='cyan', label='Volatilité (24h)')
ax1.set_title('Volatilité court terme (données horaires)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(df_bitcoin_1_day.index, df_bitcoin_1_day['volatility_30'], color='red', label='Volatilité (30j)')
ax2.set_title('Volatilité long terme (données journalières)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Les données journalières semblent présenter beaucoup moins de "bruit" que la volatilité des agrégations horaires. Il semblerait donc que l'agrégation journalière soit plus fiable pour apprécier cette tendance.

Traçons maintenant la variation relative en pourcentage du Bitcoin, en fonction des mêmes agrégations:

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

ax1.plot(df_bitcoin_1_hour.index, df_bitcoin_1_hour['returns'] * 100, color='purple', label='Variation relative horaire (%)')
ax1.set_title('Rendement court terme (données horaires)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(df_bitcoin_1_day.index, df_bitcoin_1_day['returns'] * 100, color='orange', label='Variation relative journalière (%)')
ax2.set_title('Volatilité long terme (données journalières)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Corrélations entre prix et volume en fonction de l'agrégation et de la fenêtre utilisée
Nous pouvons comparer les coefficients de corrélations entre `volume` et les différentes variables dérivées de `close`, comme `returns` ou `volatility` par exemple.

In [ ]:
df_bitcoin_1_hour['return_abs'] = df_bitcoin_1_hour['returns'].abs()
df_bitcoin_1_day['returns_abs'] = df_bitcoin_1_day['returns'].abs()

df_corr_1_hour = df_bitcoin_1_hour[['close', 'volume', 'returns', 'return_abs', 'volume_log', 'volatility_24','volatility_168']]
df_corr_1_day = df_bitcoin_1_day[['close', 'volume', 'returns', 'returns_abs', 'volume_log', 'volatility_7', 'volatility_30']]

corr_1_hour = df_corr_1_hour.corr()
corr_1_day = df_corr_1_day.corr()

In [ ]:
fig = px.imshow(
    corr_1_hour,
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    title="Heatmap de la corrélation entre variables de l'agrégation 1 heure"
)

fig.show()

In [ ]:
fig = px.imshow(
    corr_1_day,
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    title="Heatmap de la corrélation entre variables de l'agrégation 1 jour"
)

fig.show()

In [ ]:
correlation_movement = df_bitcoin_1_day['volume'].corr(df_bitcoin_1_day['returns_abs'])

print(f"Corrélation volume-prix : {df_bitcoin_1_day['volume'].corr(df_bitcoin_1_day['close']):.3f}")
print(f"Corrélation volume-mouvement : {correlation_movement:.3f}")

On remarque une plus forte corrélation entre le volume d'échange du Bitcoin et les mouvement de prix. Cependant, cette corrélation n'est pas assez significative pour indiquer un réel impact.

# Heatmap du volume moyen d'échange et heure / jour de la semaine
A l'aide de l'agrégation horaire, nous pouvons créer une heatmap du volume moyen d'échange du Bitcoin en fonction de l'heure et du jour de la semaine.

In [ ]:
df_bitcoin_heatmap = df_bitcoin_1_hour.copy()

# Extrait l'heure et le nom du jour depuis l'index temporel.
df_bitcoin_heatmap['hour'] = df_bitcoin_heatmap.index.hour
df_bitcoin_heatmap['day_of_week'] = df_bitcoin_heatmap.index.day_name()

# Définit l'ordre chronologique des jours.
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Création d'un tableau croisé des moyenne du volume par jour vs heure.
pivot_volume = df_bitcoin_heatmap.pivot_table(
    values='volume',
    index='day_of_week',
    columns='hour',
    aggfunc='mean'
)

# Les jours sont remis dans l'ordre
pivot_volume = pivot_volume.reindex(days_order)

fig = px.imshow(pivot_volume,
                labels=dict(x="Heure du jour", y="Jour de la semaine", color="Volume moyen d\'échange"),
                x=pivot_volume.columns,
                y=pivot_volume.index,
                aspect="auto",
                title="Volume d\'échange moyen du Bitcoin par heure et jour de la semaine")

fig.update_layout(height=500, width=1200)
fig.show()


On observe un pic important du volume d'échange moyen les mardis, mercredis, jeudis (pic de la semaine, avec environ 512.1455 Bitcoin échangés en moyenne) et vendredi.

On remarque aussi une faible activité les samedis et dimanches, ainsi que les lundis matin.

Ces zones d'activité correspondent aux périodes d'ouvertures des marchés financiers, ce qui semble cohérent puisque le Bitcoin est désormais un produit financier spéculatif à part entière.

# Synthèse
- Nous avons traité le fichier brut issu de Kaggle, en identifiant notamment une période de 1160 minutes manquantes. Ce fichier présente plus de 7.3 millions de lignes de données, et une fois chargé, occupe plus de 383 Mo de mémoire vive.
- Nous avons calculé des statistiques descriptives sur l'ensemble des colonnes de notre DataFrame. Nous avons aussi calculé les Z-scores et les écarts interquartiles afin d'identifier les valeurs aberrantes.
- Ces dernières ne nécessitent pas de traitement particulier, puisque le risque de fausser les données en supprimant ou modifiant des évènements bien réels est important.
- Une méthodologie pour identifier systèmatiquement les périodes de haute volatilité (bull run de 2016 à 2018 et crash pendant la période COVID par exemple) a été utilisée.
- On peut observer sur le volume d'échange du Bitcoin un pic d'activité en février 2014, ce qui correspond au piratage de la plateforme d'échange Mt. Gox et au vol de plus de 740000 Bitcoins.
- Nous pouvons affirmer que le Bitcoin est un produit fortement volatil. Les prix peuvent évoluer très rapidement et de manière très forte.
- Il n'existe qu'une faible corrélation entre prix et volume, ce qui semble indiquer que d'autres facteurs externes rentrent en jeu.
- Une heatmap du volume moyen d'échange au cours de la semaine a été réalisée. Cette dernière montre un pic d'activité de marché au coeur de la semaine, plus précisement en fin de journée. Cela correspond aux périodes d'ouverture des places financières.

# Export de l'agrégation 1h sous format Parquet pour l'application Streamlit

In [ ]:
df_bitcoin_1_hour.to_parquet('data/clean/bitcoin_1h_engineered.parquet', engine='fastparquet', compression='zstd')
print(f"DataFrame exporté en Parquet ...")